In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
import math
import re
import datetime as dt
from bureau_fc import get_bureau_feats
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")
import copy

In [2]:
DATA_DIR = '/home/sahil/data/av/AV_LTFS3/data/'

In [3]:
train = pd.read_csv(DATA_DIR+'train_Data.csv')
test = pd.read_csv(DATA_DIR+'test_Data.csv')
bur_df = pd.read_pickle(DATA_DIR+'bureau_data.pkl')

In [4]:
train.shape,test.shape,bur_df.shape

((128655, 26), (14745, 25), (591000, 32))

In [5]:
########### WRONG SIMULATION DATE CONVERSION

# train['DisbursalDate'] = pd.to_datetime(train['DisbursalDate'])
# test['DisbursalDate'] = pd.to_datetime(test['DisbursalDate'])
# train['MaturityDAte'] = pd.to_datetime(train['MaturityDAte'])
# train['DisbursalDate'] = pd.to_datetime(train['DisbursalDate'])

train['DisbursalDate'] = pd.to_datetime(train['DisbursalDate'])
test['DisbursalDate'] = pd.to_datetime(test['DisbursalDate'])
train['MaturityDAte'] = pd.to_datetime(train['MaturityDAte'])
test['MaturityDAte'] = pd.to_datetime(test['MaturityDAte'])


bur_df['DATE-REPORTED'] = pd.to_datetime(bur_df['DATE-REPORTED'])
bur_df['dpd_strin_var'] = bur_df['dpd_string'].fillna('').apply(lambda x: [x[y-3:y] for y in range(3, len(x)+3, 3)] if x.find('E')==-1 else ['000'])

In [6]:
df = pd.concat((train,test),axis=0)
df.shape

(143400, 26)

#### create features for the period of LTFS loan present in the application data, like time to next loan, time to last loan,CIBIL fc on the tradelines originated in the period

In [7]:
bur_df = bur_df.merge(df[['ID','DisbursalDate','MaturityDAte']],on='ID')
bur_df.sort_values(['ID','DISBURSED-DT'],inplace=True)

In [8]:
(bur_df['SELF-INDICATOR']==True).value_counts(1)

False    0.694228
True     0.305772
Name: SELF-INDICATOR, dtype: float64

In [9]:
temp_df = copy.deepcopy(bur_df)
temp_df.MaturityDAte = temp_df.MaturityDAte+ dt.timedelta(days=730) ### loans post 2 years of completetion of the current one
cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)&(temp_df['DISBURSED-DT']<=temp_df.MaturityDAte)
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)

temp_df = temp_df[cond1]

In [10]:
temp_df.ID.nunique(),temp_df.shape

(139964, (362624, 35))

In [11]:
# bur_df.ID.nunique()

In [12]:
# grps = temp_df[temp_df.ID.isin(train.ID.sample(1000))].groupby('ID')
grps = temp_df.groupby('ID')

In [13]:
def get_post_feats(grp):
    id_,temp = grp
    temp.MaturityDAte = temp.MaturityDAte+ dt.timedelta(days=730) ### loans post 2 years of completetion of the current one
    cond1 = (temp['DISBURSED-DT']>=temp.DisbursalDate)&(temp['DISBURSED-DT']<=temp.MaturityDAte)
    account_df = temp[cond1]
#     account_df = temp
    return get_bureau_feats(account_df)

In [14]:
%%time
try:
    pool = Pool(8) 
    data_outputs = pool.map(get_post_feats, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()

CPU times: user 1min 8s, sys: 2.81 s, total: 1min 11s
Wall time: 9min


In [15]:
bur_feats = pd.DataFrame(data_outputs)

In [16]:
bur_feats.head()

,individual_accounts,joint_accounts,guarantor_accounts,curr_bal_grtr_0,num_accounts,mean_correctedDISBURSED-AMT/HIGH CREDIT,min_correctedDISBURSED-AMT/HIGH CREDIT,max_correctedDISBURSED-AMT/HIGH CREDIT,std_correctedDISBURSED-AMT/HIGH CREDIT,sum_correctedDISBURSED-AMT/HIGH CREDIT,mean_correctedCURRENT-BAL,min_correctedCURRENT-BAL,max_correctedCURRENT-BAL,std_correctedCURRENT-BAL,sum_correctedCURRENT-BAL,mean_correctedOVERDUE-AMT,min_correctedOVERDUE-AMT,max_correctedOVERDUE-AMT,std_correctedOVERDUE-AMT,sum_correctedOVERDUE-AMT,num_closed_accounts,num_open_accounts,num_delinq_accounts,total_written_off_amount,mean_percent_paid_off,min_percent_paid_off,max_percent_paid_off,std_percent_paid_off,overall_percent_paid_off,median_tenor,max_tenor,min_tenor,std_count,ddd_count,xxx_count,late_count,_30_count,_60_count,_90_count,_180_count,total_count,std_count_application_loan,ddd_count_application_loan,xxx_count_application_loan,late_count_application_loan,_30_count_application_loan,_60_count_application_loan,_90_count_application_loan,_180_count_application_loan,total_count_application_loan,mean_dpd_str,min_dpd_str,max_dpd_str,std_dpd_str,mean_dpd_str_application_loan,min_dpd_str_application_loan,max_dpd_str_application_loan,std_dpd_str_application_loan,num_accounts__between_0_and_365_days,total_sanctioned_amount__between_0_and_365_days,total_curr_bal__between_0_and_365_days,overall_percentage_paid_off__between_0_and_365_days,num_accounts__between_365_and_730_days,total_sanctioned_amount__between_365_and_730_days,total_curr_bal__between_365_and_730_days,overall_percentage_paid_off__between_365_and_730_days,num_accounts__between_730_and_1095_days,total_sanctioned_amount__between_730_and_1095_days,total_curr_bal__between_730_and_1095_days,overall_percentage_paid_off__between_730_and_1095_days,num_accounts__between_1095_and_1460_days,total_sanctioned_amount__between_1095_and_1460_days,total_curr_bal__between_1095_and_1460_days,overall_percentage_paid_off__between_1095_and_1460_days,num_accounts__between_1460_and_3650_days,total_sanctioned_amount__between_1460_and_3650_days,total_curr_bal__between_1460_and_3650_days,overall_percentage_paid_off__between_1460_and_3650_days,num_accounts_Tractor Loan,total_sanctioned_amount_Tractor Loan,total_curr_bal_Tractor Loan,overall_percentage_paid_off_Tractor Loan,num_accounts_Gold Loan,total_sanctioned_amount_Gold Loan,total_curr_bal_Gold Loan,overall_percentage_paid_off_Gold Loan,num_accounts_Business Loan Priority Sector Agriculture,total_sanctioned_amount_Business Loan Priority Sector Agriculture,total_curr_bal_Business Loan Priority Sector Agriculture,overall_percentage_paid_off_Business Loan Priority Sector Agriculture,num_accounts_Kisan Credit Card,total_sanctioned_amount_Kisan Credit Card,total_curr_bal_Kisan Credit Card,overall_percentage_paid_off_Kisan Credit Card,num_accounts_Auto Loan (Personal),total_sanctioned_amount_Auto Loan (Personal),total_curr_bal_Auto Loan (Personal),overall_percentage_paid_off_Auto Loan (Personal),num_accounts_Personal Loan,total_sanctioned_amount_Personal Loan,total_curr_bal_Personal Loan,overall_percentage_paid_off_Personal Loan,num_accounts_Other,total_sanctioned_amount_Other,total_curr_bal_Other,overall_percentage_paid_off_Other,num_accounts_Overdraft,total_sanctioned_amount_Overdraft,total_curr_bal_Overdraft,overall_percentage_paid_off_Overdraft,mean_day_start_day_diff_app_vs_other,min_day_start_day_diff_app_vs_other,max_day_start_day_diff_app_vs_other,std_day_start_day_diff_app_vs_other,mean_days_bw_loans,min_days_bw_loans,max_days_bw_loans,std_days_bw_loans,sum_days_bw_loans,ID
0,6,0,0,1,6,252058.666667,0.0,500000.0,1.978983e+05,1512352.0,6312.166667,0.0,37873.0,1.546159e+04,37873.0,7574.6,0.0,37873.0,16937.320502,37873.0,5,0,1,0.0,0.797210,-0.013948,1.0,0.453452,0.974958,1472.0,1704.0,109.0,72,3,0,74,5,1,0,0,149,7,0,0,4,2,2,2,0,35,12.919463,0,87,14.289677,0.000000,0,0,0.000000,2,775000.0,0.0,1.000000,2,700000.0,0.0,1.000000,0,0.0,0.0,NaN,1,37352.0,37873.0,-0

In [17]:
bur_feats.shape

(139964, 120)

In [18]:
bur_feats.to_pickle('bureau_future_feats_start_2_maturity_plus_2.pkl')

#### ltfs loan variables

In [14]:
temp_df = copy.deepcopy(bur_df)
temp_df.MaturityDAte = temp_df.MaturityDAte+ dt.timedelta(days=730) ### loans post 2 years of completetion of the current one
cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)&(temp_df['DISBURSED-DT']<=temp_df.MaturityDAte)
cond2 = bur_df['SELF-INDICATOR']==True
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)

temp_df = temp_df[cond1&cond2]

In [15]:
temp_df.ID.nunique(),temp_df.shape

(137534, (166712, 35))

In [16]:
# grps = temp_df[temp_df.ID.isin(train.ID.sample(1000))].groupby('ID')
grps = temp_df.groupby('ID')

In [17]:
%%time
try:
    pool = Pool(8) 
    data_outputs = pool.map(get_post_feats, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()

CPU times: user 1min 5s, sys: 2.25 s, total: 1min 7s
Wall time: 8min 39s


In [18]:
ltfs_bur_feats = pd.DataFrame(data_outputs)
ltfs_bur_feats.shape

(137534, 120)

In [19]:
ltfs_bur_feats.columns = ['ltfs_'+x if x!='ID' else x for x in ltfs_bur_feats.columns]

In [28]:
ltfs_bur_feats.to_pickle('ltfs_bureau_future_feats_start_2_maturity_plus_2.pkl')